In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Importing the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import accuracy

In [ ]:
train = pd.read_csv('/kaggle/input/ea-predict-dataset/train.csv')
movies = pd.read_csv('/kaggle/input/ea-predict-dataset/movies.csv')
test = pd.read_csv('/kaggle/input/ea-predict-dataset/test.csv')

In [ ]:
#viewing training data
train.head()

In [ ]:
#viewing Movie data
movies.head()

In [ ]:
#viewing test data
test.head()

In [ ]:
train_combined = pd.merge(train, movies, on='movieId')

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(x='rating', data= train_combined )
plt.title(' Ratings')
plt.show()

In [ ]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(train_combined[['userId', 'movieId', 'rating']], reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2)

In [ ]:

from surprise import SVD
#  model
algo = SVD(n_epochs = 40, n_factors = 1200, init_std_dev = 0.005, random_state = 400, init_mean = 0.005)
algo.fit(trainset)

In [ ]:
rating_predict = algo.test(testset)
 
# model Evaluation
accuracy.rmse(rating_predict)
 
# Print RMSE
print("rsme: ", accuracy.rmse(rating_predict, verbose=False))

In [ ]:
test_combined = pd.merge(test, movies, on='movieId')

In [ ]:
rate_predict = [algo.predict(uid, iid) for uid, iid in test_combined[['userId', 'movieId']].values]

In [ ]:
submission = pd.DataFrame(rate_predict, columns=['uid', 'iid', 'r_ui', 'est', 'details'])
submission['Id'] = submission[['uid', 'iid']].apply(lambda x: f'{x[0]}_{x[1]}', axis=1)
submission['rating'] = submission['est']
submission[['Id', 'rating']].to_csv('model_svd.csv', index=False)